In [ ]:
# !pip install --progress-bar off --quiet manifoldpy 
# !pip install --upgrade --progress-bar off --quiet https://github.com/vluzko/manifoldpy/tarball/main
!pip install -r requirements.txt

  Cloning https://github.com/habakukhain/manifoldpy (to revision post_comment) to /private/var/folders/xv/wwxc1s9943185lf_x0py59dh0000gn/T/pip-req-build-w_gdrkx8
  Running command git clone --filter=blob:none --quiet https://github.com/habakukhain/manifoldpy /private/var/folders/xv/wwxc1s9943185lf_x0py59dh0000gn/T/pip-req-build-w_gdrkx8
  Running command git checkout -b post_comment --track origin/post_comment
  Switched to a new branch 'post_comment'
  Branch 'post_comment' set up to track remote branch 'post_comment' from 'origin'.
  Resolved https://github.com/habakukhain/manifoldpy to commit b49f57bb5587459422b977887e95be102d2a1860
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.23.1-cp39-cp39-macosx_10_9_x86_64.whl (18.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:


In [7]:
from manifoldpy import api
import json
from os import path
import requests
import pandas as pd

In [8]:
def get_all_markets():
    markets = api._get_all_markets()
    with open("manifold_data/all-manifold-markets.json","w",encoding="utf-8",) as f:
        json.dump(markets, f, ensure_ascii=False, indent=4)
    with open("manifold_data/all-manifold-markets.json", "r", encoding="utf-8") as f:
        markets = json.load(f)
    markets = pd.DataFrame(markets).set_index('id')
    return markets

def load_markets():
    with open("manifold_data/all-manifold-markets.json", "r", encoding="utf-8") as f:
        markets = json.load(f)
    markets = pd.DataFrame(markets).set_index('id')
    return markets

def get_user_id(username: str) -> str:
    user_id=api.get_user_by_name(username).id
    return user_id

def get_market_description(markets):
    import re
    for id in markets.index:
        url = "https://manifold.markets/api/v0/market/" + id
        market_info = requests.get(url, timeout=20).json()
        desc = str(market_info['textDescription'])
        url = re.search("(www|http:|https:)+[^\s]+[\w]", desc)
        metaculus_id = (str(url).split('/')[4])
        markets.loc[id,'textDescription'] = desc
        markets.loc[id,'metaculusId'] = metaculus_id
    return markets

def get_metaculus_predictions(markets):
    i = 0
    for id in markets.index:
        url = "https://www.metaculus.com/api2/questions/" + markets.loc[id, 'metaculusId']
        metaculus_info = requests.get(url, timeout=20).json()
        metaculus_type = metaculus_info['possibilities']['type']
        if metaculus_type != 'binary':
            markets = markets.drop(id)
            continue
        metaculus_prediction = metaculus_info['prediction_timeseries'][-1]['community_prediction']
        manifold_prediction = markets.loc[id, 'probability']
        markets.loc[id,'metaculusPrediction'] = metaculus_prediction
        markets.loc[id, 'diff'] = metaculus_prediction - manifold_prediction
    return markets

def decider(market, lim):
    diff = market['diff']
    if abs(diff) < lim:
        outcome = 'SELL'
    elif diff < 0: # Metaculus predicts higher then Manifold, you should buy YES
        outcome = 'NO'
    elif diff > 0: 
        outcome = 'YES'
    return outcome

def allocator (markets, lim):
    trading_markets = markets[abs(markets['diff']) > lim]
    sum_diff = sum(abs(trading_markets['diff']))
    for id in markets.index:
        if abs(markets.loc[id]['diff']) < lim:
            markets.loc[id,'weights'] = 0
        else:
            markets.loc[id,'weights'] = abs(markets.loc[id,'diff'])/sum_diff
    return markets

In [9]:
manifold_key = 'key' 
wrapper = api.APIWrapper(manifold_key)

In [10]:
reload_markets = False
if reload_markets:
    markets = get_all_markets()
else: 
    markets = load_markets()

metaculus_markets = markets[markets["creatorId"] == "jn8ZKjXgwgfbwJqHRNbemA3Epw52"]
metaculus_markets = metaculus_markets[metaculus_markets["outcomeType"] == "BINARY"]
metaculus_markets = metaculus_markets[metaculus_markets["isResolved"] == False]
metaculus_markets = get_market_description(metaculus_markets)
metaculus_markets = get_metaculus_predictions(metaculus_markets)

In [83]:
playmoney = 50
lim = 0.03
metaculus_markets = allocator(metaculus_markets,lim)

for id in metaculus_markets.index:
    q = metaculus_markets.loc[id]
    q_text = q['question']
    metaculus_score = q['metaculusPrediction']
    manifold_score = q['probability']
    outcome = decider(q, lim)
    betsize = q['weights']*playmoney
    if outcome == 'SELL':
        wrapper.sell_shares(id)
        print(f'The question was \"{q_text}\",\n Metaculus predicted {metaculus_score:.3f} and Manifold predicted {manifold_score:.3f}. \n The difference was <{lim*100}% \n I sold my position \n \n')
        #message = f'This is an attempt to make use of [Nikos Bosse\'s observation](https://forum.effectivealtruism.org/posts/PGqu4MD3AKHun7kaF/predictive-performance-on-metaculus-vs-manifold-markets) that Metaculus systematically outperforms Manifold markets. Metaculus predicted {metaculus_score:.3f}. The difference was <{lim*100}%, so sold my position (if I had a position).'
        continue
    else: 
        wrapper.make_bet(betsize, id, outcome)
        print(f'I made a bet on manifold markets! \n The question was \"{q_text}\" \n Metaculus predicted {metaculus_score:.3f} and Manifold predicted {manifold_score:.3f}. \n To correct the market on manifold I bet {betsize:.2f} Mana on {outcome} \n \n')
        message = f'This is an attempt to make use of [Nikos Bosse\'s observation](https://forum.effectivealtruism.org/posts/PGqu4MD3AKHun7kaF/predictive-performance-on-metaculus-vs-manifold-markets) that Metaculus systematically outperforms Manifold markets. [Metaculus](https://www.metaculus.com/questions/{q['metaculusId']}) predicted {metaculus_score:.3f}. To correct the market on manifold I automatically bet {betsize:.2f} M$ on {outcome}'
        wrapper.make_comment(id, message)
        continue

I made a bet on manifold markets! 
 The question was "Will Ukraine sever the land bridge between Crimea and Russia before 2024?" 
 Metaculus predicted 0.270 and Manifold predicted 0.333. 
 To correct the market on manifold I bet 1.95 Mana on NO 
 

I made a bet on manifold markets! 
 The question was "Will the US enter a recession by January 1, 2023?" 
 Metaculus predicted 0.250 and Manifold predicted 0.119. 
 To correct the market on manifold I bet 4.05 Mana on YES 
 

I made a bet on manifold markets! 
 The question was "Will Hunter Biden be indicted before November 5, 2024?" 
 Metaculus predicted 0.350 and Manifold predicted 0.383. 
 To correct the market on manifold I bet 1.02 Mana on NO 
 

I made a bet on manifold markets! 
 The question was "When the next referendum is held on Scotland's Union with the United Kingdom, will Scotland vote to leave?",
 Metaculus predicted 0.560 and Manifold predicted 0.554. 
 The difference was <3.0% 
 I sold my position 
 

I made a bet on manifol

NameError: name 'metaculus_markets' is not defined